In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import sys
sys.path.append('..')

In [23]:
from hdna import * 
import nupack as nu
import networkx as nx 
import numpy as np
from fitter import Fitter
from hdna.reporting import histotime as ht

In [24]:
HP = {
    #model free parameters  
    'minimum_nucleation': 1,
    'sliding_cutoff':     200,
    'sliding_filter':     4,
    'zipping_rate':       2e7,
    'sliding_rate':       5e6,
    #temperature
    'temperature':        25,       #### HERTEL EXPERIMENTAL TEMPERATURE 
    #angles
    'azimutal_angle':     120,
    'longitudinal_angle': 270,
}

OPT = {
    #simulation options
    'runtime': 4e-6,
    'N_simul': 1000,
    'trajstosave': 35,
    #nupack options
    'stacking': 'nostacking'
}

SOPT = {    
    #datasaving options 
    'G_saving': 'strand_folder'
}

# Actual computation 
MOD = Model('dna', '3D', 
        min_nucleation=HP['minimum_nucleation'], 
        sliding_cutoff=HP['sliding_cutoff'],
        sliding_filter=HP['sliding_filter'],
        stacking=OPT['stacking'],
        sliding=HP['sliding_rate'],
        zipping=HP['zipping_rate'],
        celsius=HP['temperature'])

In [25]:
s1 = Strand(MOD, "AAACCACACA")
s2 = s1.complementary()
C = Chamber(MOD, s1, s2)
N = Kinetwork(MOD, s1, s2, Geometry(120,270))
OZZ = Options(Nsim=10000)
S = Simulator(MOD, N, OZZ)

In [26]:
sim = S.ensemble()

100%|██████████| 10000/10000 [00:03<00:00, 3322.27it/s]


In [27]:
tau = S.fpts(sim)

0 simulations didn't produce a duplex.
That's 0.0% of simulations


In [41]:
from scipy.stats import gamma, expon

In [42]:
mp = S.mfpt(sim)
fit_alpha, fit_loc, fit_beta = gamma.fit(tau)

exploc, expscale = expon.fit(tau)

0 simulations didn't produce a duplex.
That's 0.0% of simulations


In [48]:
exploc, expscale

(6.117344399568561e-09, 3.342335086831045e-07)

In [46]:
rv = gamma(a=fit_alpha, loc=fit_loc, scale=fit_beta)
rve = expon(loc=exploc*10, scale=expscale)

In [44]:
mod = 1/rv.mean()
ht(tau, rv, OZZ.runtime, exp=8,mod=mod)

In [47]:
mod = 1/rve.mean()
ht(tau, rve, OZZ.runtime, exp=8,mod=mod)

In [37]:
import plotly.graph_objects as go
import numpy as np 
nbins = 150
X = np.linspace(0,OZZ.runtime,500)
# Y = [gamma.pdf(j, a=fit_alpha, scale=fit_beta) for j in X]
data1 = go.Scatter(x=X, y=gamma.pdf(X, a=fit_alpha, scale=fit_beta))
data2 = go.Histogram(x=tau, histnorm='probability density', nbinsx=nbins)

layout = go.Layout(xaxis=dict(tickformat=".0e"),
                   yaxis=dict(tickformat=".0e"))

fig = go.Figure(data=[data1, data2], layout=layout)
fig.update_traces(opacity=0.85)
fig.show()

In [38]:
import openturns as ot 

fitgamma = ot.GammaFactory().build(tau)

TypeError: InvalidArgumentException : Error: cannot build a Gamma distribution from the given parameters